In [1]:
%pip install rdflib>=6.3

zsh:1: 6.3 not found
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from pathlib import Path
import json
import re
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, XSD, OWL


EX = Namespace("http://example.org/exercise#")
SULO = Namespace("https://w3id.org/sulo/")
TIME = Namespace("http://www.w3.org/2006/time#")


ATTR_CLASSES = {
    "ImpactLevel": EX.ImpactLevelQuality,
    "Intensity": EX.IntensityQuality,
    "BalanceRequirement": EX.BalanceRequirementQuality,
    "JointInvolvement": EX.JointInvolvementQuality,
}

# Fitness guideline classes
FITNESS_CLASSES = {
    "Frequency": EX.FrequencyQuality,
    "Intensity": ATTR_CLASSES["Intensity"],  
    "Duration": EX.DurationQuality,
    "Type": EX.ExerciseTypeQuality,
    "RestPeriod": EX.RestPeriodQuality,
    "HeartRateZone": EX.HeartRateZoneQuality,
    "Progression": EX.ProgressionQuality, 
    "Supervision": EX.SupervisionQuality,
    "Equipment": EX.ExerciseEquipment  
}


TIME_UNITS = [
    "minute", "minutes", "hour", "hours", "day", "days", 
    "week", "weeks", "month", "months", "year", "years",
    "per day", "per week", "per month"
]

def slug(text: str) -> str:
    """Safe URI suffix."""
    return re.sub(r'[^A-Za-z0-9]+', '_', text.strip()).strip('_')

def ensure(g: Graph, uri: URIRef, rdf_type: URIRef, label: str | None = None):
    """Ensure a URI exists in the graph with given type and label."""
    if (uri, RDF.type, None) not in g:
        g.add((uri, RDF.type, rdf_type))
    if label:
        g.add((uri, RDFS.label, Literal(label, lang='en')))

def ensure_units(g):
    """Ensure all time units exist in the ontology."""
 
    ensure(g, SULO.Unit, OWL.Class, "Unit")
    
 
    ensure(g, EX.TimeUnit, OWL.Class, "Time Unit")
    g.add((EX.TimeUnit, RDFS.subClassOf, SULO.Unit))
    

    ensure(g, EX.FrequencyUnit, OWL.Class, "Frequency Unit")
    g.add((EX.FrequencyUnit, RDFS.subClassOf, SULO.Unit))
    

    for unit in TIME_UNITS:
        unit_uri = EX[f"{slug(unit)}Unit"]
        
        # Determine if it's a time unit or frequency unit
        if unit.startswith("per "):
            ensure(g, unit_uri, EX.FrequencyUnit, unit)
        else:
            ensure(g, unit_uri, EX.TimeUnit, unit)

def inject_fitness_guidelines(g, guidelines):
    """
    Add fitness guidelines to an existing ontology graph.
    Guidelines are linked to health conditions, not to specific variations.
    Reuses existing SULO classes like InformationObject, Quantity, Duration, etc.
    """

    ensure(g, EX.FitnessGuidelineQuality, OWL.Class, "Fitness guideline quality")
    g.add((EX.FitnessGuidelineQuality, RDFS.subClassOf, SULO.Quality))
    

    ensure(g, SULO.Capability, OWL.Class, "Capability")
    g.add((SULO.Capability, RDFS.comment, Literal("A capability is a feature that describes what an entity is able to do, under some set of circumstances.", lang='en')))

    ensure(g, SULO.Recommendation, OWL.Class, "Recommendation")
    g.add((SULO.Recommendation, RDFS.subClassOf, SULO.Capability))
    g.add((SULO.Recommendation, RDFS.comment, Literal("A recommendation is a capability that suggests what an entity should do under certain health conditions.", lang='en')))
    

    ensure(g, SULO.SpatialObject, OWL.Class, "Spatial Object")
    ensure(g, EX.ExerciseEquipment, OWL.Class, "Exercise Equipment")
    g.add((EX.ExerciseEquipment, RDFS.subClassOf, SULO.SpatialObject))
    

    for guideline_name, guideline_class in FITNESS_CLASSES.items():
        if guideline_name != "Equipment": 
            ensure(g, guideline_class, OWL.Class, f"{guideline_name} quality")
            g.add((guideline_class, RDFS.subClassOf, EX.FitnessGuidelineQuality))
    

    ensure_units(g)
    

    condition_guidelines = {}
    for guideline in guidelines:
        cond_label = guideline["HealthCondition"].strip()
        if cond_label not in condition_guidelines:
            condition_guidelines[cond_label] = []
        condition_guidelines[cond_label].append(guideline)
    

    for cond_label, cond_guidelines in condition_guidelines.items():

        cond_class = EX[f"{slug(cond_label)}Quality"]
        

        guideline_set_uri = EX[f"GuidelineSet_{slug(cond_label)}"]
        ensure(g, guideline_set_uri, SULO.Set, f"fitness guidelines for {cond_label}")
        

        g.add((cond_class, SULO.refersTo, guideline_set_uri))
        
   
        for i, guideline in enumerate(cond_guidelines):

            rec_number = i + 1
            rec_uri = EX[f"Recommendation_{slug(cond_label)}_{rec_number}"]
            ensure(g, rec_uri, SULO.Recommendation, f"Fitness recommendation {rec_number} for {cond_label}")
            

            g.add((guideline_set_uri, SULO.hasPart, rec_uri))
            

            
            if "Frequency" in guideline:
                freq_value = guideline["Frequency"]["Value"]
                freq_unit = guideline["Frequency"]["Unit"]
                

                freq_quantity_uri = EX[f"FrequencyQuantity_{slug(cond_label)}_{rec_number}"]
                ensure(g, freq_quantity_uri, SULO.Quantity, f"Frequency quantity: {freq_value} {freq_unit}")
                
   
                g.add((freq_quantity_uri, SULO.hasMagnitude, Literal(freq_value, datatype=XSD.decimal)))
                unit_uri = EX[f"{slug(freq_unit)}Unit"]
                g.add((freq_quantity_uri, SULO.hasUnit, unit_uri))
                

                freq_info_uri = EX[f"FrequencyInfo_{slug(cond_label)}_{rec_number}"]
                ensure(g, freq_info_uri, SULO.InformationObject, f"Frequency: {freq_value} {freq_unit}")
                

                g.add((freq_info_uri, SULO.represents, freq_quantity_uri))
                g.add((freq_info_uri, SULO.refersTo, FITNESS_CLASSES["Frequency"]))
                g.add((rec_uri, SULO.refersTo, freq_info_uri))
            

            if "Intensity" in guideline:
                intensity_value = guideline["Intensity"]
                
                intensity_info_uri = EX[f"IntensityInfo_{slug(cond_label)}_{slug(intensity_value)}_{rec_number}"]
                ensure(g, intensity_info_uri, SULO.InformationObject, f"Intensity: {intensity_value}")

                g.add((intensity_info_uri, SULO.hasValue, Literal(intensity_value, datatype=XSD.string)))
                g.add((intensity_info_uri, SULO.refersTo, FITNESS_CLASSES["Intensity"]))

                g.add((rec_uri, SULO.refersTo, intensity_info_uri))
            

            if "Duration" in guideline:
                duration_value = guideline["Duration"]["Value"]
                duration_unit = guideline["Duration"]["Unit"]
                

                duration_quantity_uri = EX[f"DurationQuantity_{slug(cond_label)}_{rec_number}"]
                ensure(g, duration_quantity_uri, SULO.Duration, f"Duration: {duration_value} {duration_unit}")
                

                g.add((duration_quantity_uri, SULO.hasMagnitude, Literal(duration_value, datatype=XSD.decimal)))
                

                unit_uri = EX[f"{slug(duration_unit)}Unit"]
                g.add((duration_quantity_uri, SULO.hasUnit, unit_uri))
                
                duration_info_uri = EX[f"DurationInfo_{slug(cond_label)}_{rec_number}"]
                ensure(g, duration_info_uri, SULO.InformationObject, f"Duration: {duration_value} {duration_unit}")
                

                g.add((duration_info_uri, SULO.represents, duration_quantity_uri))
                g.add((duration_info_uri, SULO.refersTo, FITNESS_CLASSES["Duration"]))

                g.add((rec_uri, SULO.refersTo, duration_info_uri))
            

            if "Type" in guideline:
                type_value = guideline["Type"]
                

                if isinstance(type_value, list):
                    for exercise_type in type_value:

                        type_info_uri = EX[f"ExerciseTypeInfo_{slug(cond_label)}_{slug(exercise_type)}_{rec_number}"]
                        ensure(g, type_info_uri, SULO.InformationObject, f"Exercise type: {exercise_type}")
                        

                        g.add((type_info_uri, SULO.hasValue, Literal(exercise_type, datatype=XSD.string)))
                        g.add((type_info_uri, SULO.refersTo, FITNESS_CLASSES["Type"]))
                        
                        g.add((rec_uri, SULO.refersTo, type_info_uri))
                else:
                    type_info_uri = EX[f"ExerciseTypeInfo_{slug(cond_label)}_{slug(type_value)}_{rec_number}"]
                    ensure(g, type_info_uri, SULO.InformationObject, f"Exercise type: {type_value}")

                    g.add((type_info_uri, SULO.hasValue, Literal(type_value, datatype=XSD.string)))
                    g.add((type_info_uri, SULO.refersTo, FITNESS_CLASSES["Type"]))

                    g.add((rec_uri, SULO.refersTo, type_info_uri))

            if "RestPeriod" in guideline:
                rest_value = guideline["RestPeriod"]
                
                if isinstance(rest_value, dict) and "Value" in rest_value and "Unit" in rest_value:
                    rest_value_num = rest_value["Value"]
                    rest_unit = rest_value["Unit"]

                    rest_quantity_uri = EX[f"RestPeriodQuantity_{slug(cond_label)}_{rec_number}"]
                    ensure(g, rest_quantity_uri, SULO.Duration, f"Rest period: {rest_value_num} {rest_unit}")

                    g.add((rest_quantity_uri, SULO.hasMagnitude, Literal(rest_value_num, datatype=XSD.decimal)))
                    

                    unit_uri = EX[f"{slug(rest_unit)}Unit"]
                    g.add((rest_quantity_uri, SULO.hasUnit, unit_uri))
                    
                    rest_info_uri = EX[f"RestPeriodInfo_{slug(cond_label)}_{rec_number}"]
                    ensure(g, rest_info_uri, SULO.InformationObject, f"Rest period: {rest_value_num} {rest_unit}")
                    
                    g.add((rest_info_uri, SULO.represents, rest_quantity_uri))
                    g.add((rest_info_uri, SULO.refersTo, FITNESS_CLASSES["RestPeriod"]))
                    g.add((rec_uri, SULO.refersTo, rest_info_uri))
                else:

                    rest_info_uri = EX[f"RestPeriodInfo_{slug(cond_label)}_{slug(str(rest_value))}_{rec_number}"]
                    ensure(g, rest_info_uri, SULO.InformationObject, f"Rest period: {rest_value}")
                    
                    g.add((rest_info_uri, SULO.hasValue, Literal(str(rest_value), datatype=XSD.string)))
                    g.add((rest_info_uri, SULO.refersTo, FITNESS_CLASSES["RestPeriod"]))
                    g.add((rec_uri, SULO.refersTo, rest_info_uri))
            
            for attr in ["HeartRateZone", "Progression", "Supervision"]:
                if attr in guideline:
                    attr_value = guideline[attr]
                    
                    if isinstance(attr_value, list):
                        for val in attr_value:
                            attr_info_uri = EX[f"{attr}Info_{slug(cond_label)}_{slug(val)}_{rec_number}"]
                            ensure(g, attr_info_uri, SULO.InformationObject, f"{attr}: {val}")
                            
                            g.add((attr_info_uri, SULO.hasValue, Literal(val, datatype=XSD.string)))
                            g.add((attr_info_uri, SULO.refersTo, FITNESS_CLASSES[attr]))

                            g.add((rec_uri, SULO.refersTo, attr_info_uri))
                    else:
                        attr_info_uri = EX[f"{attr}Info_{slug(cond_label)}_{slug(attr_value)}_{rec_number}"]
                        ensure(g, attr_info_uri, SULO.InformationObject, f"{attr}: {attr_value}")
                        
                        g.add((attr_info_uri, SULO.hasValue, Literal(attr_value, datatype=XSD.string)))
                        g.add((attr_info_uri, SULO.refersTo, FITNESS_CLASSES[attr]))
                        
                        g.add((rec_uri, SULO.refersTo, attr_info_uri))
            
            if "Equipment" in guideline:
                equipment_value = guideline["Equipment"]
                
                if isinstance(equipment_value, list):
                    for equipment in equipment_value:
                        equipment_uri = EX[f"Equipment_{slug(equipment)}"]
                        ensure(g, equipment_uri, EX.ExerciseEquipment, equipment)

                        equip_info_uri = EX[f"EquipmentInfo_{slug(cond_label)}_{slug(equipment)}_{rec_number}"]
                        ensure(g, equip_info_uri, SULO.InformationObject, f"Equipment: {equipment}")

                        g.add((equip_info_uri, SULO.refersTo, equipment_uri))
                        g.add((rec_uri, SULO.refersTo, equip_info_uri))
                else:
                    equipment_uri = EX[f"Equipment_{slug(equipment_value)}"]
                    ensure(g, equipment_uri, EX.ExerciseEquipment, equipment_value)
                    
                    equip_info_uri = EX[f"EquipmentInfo_{slug(cond_label)}_{slug(equipment_value)}_{rec_number}"]
                    ensure(g, equip_info_uri, SULO.InformationObject, f"Equipment: {equipment_value}")

                    g.add((equip_info_uri, SULO.refersTo, equipment_uri))
                    g.add((rec_uri, SULO.refersTo, equip_info_uri))

def inject_fitness_guidelines_multi(schema_ttl: Path, out_ttl: Path, *json_files: Path):
    """
    Merge fitness guidelines from JSON files 
    """
    g = Graph()
    g.parse(schema_ttl, format="turtle")
    
    all_guidelines = []
    for jfile in json_files:
        with open(jfile, encoding="utf-8") as fh:
            all_guidelines.extend(json.load(fh))
    
    inject_fitness_guidelines(g, all_guidelines)
    g.serialize(out_ttl, format="turtle")
    print(f"Saved merged ontology with fitness guidelines to {out_ttl}")

if __name__ == "__main__":
    schema_path = Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/sulo_2.ttl')
    out_path = Path('sulo_with_guidelines.ttl')
    guidelines_paths = [

        Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/Guidelines/amputees_guidelines.json'),
        Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/Guidelines/arthritis_guidelines.json'),
        Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/Guidelines/cerebral_palsy_guidelines.json'),
        Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/Guidelines/chronic_back_pain.json'),
        Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/Guidelines/chronic_fatigue_sundrome_guidelines.json'),
        Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/Guidelines/diabetes_guidelines.json'),
        Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/Guidelines/fibromyalgia_guidelines.json'),
        Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/Guidelines/frailty_guidelines.json'),
        Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/Guidelines/multiple_sclerosis_guidelines.json'),
        Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/Guidelines/muscular_dystrophy.json'),
        Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/Guidelines/myasthenia_gravis_guidelines.json'),
        Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/Guidelines/myopathy_guidelines.json'),
        Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/Guidelines/obesity_guidelines.json'),
        Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/Guidelines/osteoporosis_guidelines.json'),
        Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/Guidelines/paraplegia_wheelchair-bound_guidelines.json'),
        Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/Guidelines/parkinson_disease_guidelines.json'),
        Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/Guidelines/peripheral_neuropathy_guidelines.json'),
        Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/Guidelines/spinal_cord_injury_guidelines.json'),
        Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/Guidelines/stroke_guidelines.json'),
        Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/Guidelines/traumatic_brain_injury_guidelines.json'),
        Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/Guidelines/wheelchair_use.json'),

    ]
    

    inject_fitness_guidelines_multi(schema_path, out_path, *guidelines_paths)

✓ Saved merged ontology with fitness guidelines to sulo_with_guidelines.ttl
